# 翻譯

與其他人交流的能力是使人類文明得以發展的驅動力之一。在多數人的努力嘗試中，交流是關鍵。

![一個掌握多種語言的傀儡程式](./images/translation.jpg)

人工智慧 (AI) 能透過翻譯不同語言間的文字或語音協助簡化交流，還能協助打破跨越國家/地區和文化的交流屏障。

## 建立認知服務資源

在 Azure 中，您可以使用認知服務在多個語言間的進行翻譯。

如果您還沒有該資源，請使用以下步驟在您的 Azure 訂用帳戶中建立一個**認知服務**資源：

> **備註**：若您已經有認知服務資源，只需在 Azure 入口網站中開啟其 **[快速入門]**頁面並將其金鑰和端點複製到下面的儲存格。否則，可追隨下面的步驟來建立一個認知服務資源。

1.在其它瀏覽器索引標籤中，透過 https://portal.azure.com 開啟 Azure 入口網站，並用您的 Microsoft 帳戶登入。
2.按一下 **[&#65291; 建立資源]** 按鈕，搜尋*認知服務*，並建立包含以下設定的**認知服務**資源：
    - **訂用帳戶**： *您的 Azure 訂用帳戶*。 
    - **資源群組**： *選取或建立具有唯一名稱的資源群組*。 
    - **區域**： *選擇任一可用區域*： 
    - **名稱**： *輸入唯一名稱*。 
    - **定價層** ：S0
    - **我確認已閱讀通知並理解通知內容**： 已選取。
3.等待部署完成。然後前往您的認知服務資源，在 **[概觀]** 頁面上，按一下連結以管理服務金鑰。您將需要端點和金鑰，以便從用戶端應用程式連線到您的認知服務資源。

### 獲取適用於認知服務資源的金鑰和位置

若要使用您的認知服務資源，用戶端應用程式需要其驗證金鑰和位置：

1.在 Azure 入口網站中，您的認知服務資源之 **[金鑰和端點]** 頁面上，複製您的資源之**金鑰 1**並將其貼上到下面的程式碼，取代 **YOUR_COG_KEY**。
2.為您的資源複製**位置**並將其貼上到下方程式碼中，取代 **YOUR_COG_LOCATION**。
>**備註**：留在 **[金鑰和端點]** 頁面並從此頁面複製**位置**(範例：_westus_)。請 _勿_ 在 [位置] 欄位的文字之間增加空格。 
3.透過按一下儲存格左側的 **[執行儲存格]** (&#9655;) 按鈕執行下方程式碼。

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_location = 'YOUR_COG_LOCATION'

print('Ready to use cognitive services in {} using key {}'.format(cog_location, cog_key))

## 翻譯文字

顧名思義，**翻譯文字**服務可以讓您將文字從一種語言翻譯為另一種語言。

沒有適用於此服務的 Python SDK，但您可以使用它的 REST 介面透過 HTTP 向端點提交要求，這在 Python 中透過使用**要求**程式庫相對容易操作。關於待翻譯文字和產生的已翻譯文字之資訊以 JSON 格式交換。

執行下列儲存格以建立執行此操作的函式，然後用一個從英文到法文的簡單翻譯來測試此函式。

In [ ]:
# Create a function that makes a REST request to the Text Translation service
def translate_text(cog_location, cog_key, text, to_lang='fr', from_lang='en'):
    import requests, uuid, json

    # Create the URL for the Text Translator service REST request
    path = 'https://api.cognitive.microsofttranslator.com/translate?api-version=3.0'
    params = '&from={}&to={}'.format(from_lang, to_lang)
    constructed_url = path + params

    # Prepare the request headers with Cognitive Services resource key and region
    headers = {
        'Ocp-Apim-Subscription-Key': cog_key,
        'Ocp-Apim-Subscription-Region':cog_location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    # Add the text to be translated to the body
    body = [{
        'text': text
    }]

    # Get the translation
    request = requests.post(constructed_url, headers=headers, json=body)
    response = request.json()
    return response[0]["translations"][0]["text"]


# Test the function
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='fr', from_lang='en')
print('{} -> {}'.format(text_to_translate,translation))

此服務應將英文文字 "Hello" 翻譯為法文 "Bonjour"。

注意，這些語言透過使用語言縮寫標準體系來指定，*en* 表示英文，*fr* 表示法文。您也可以使用包含特定文化的縮寫，當同一語言用於不同地理區域時 (通常拼字不同)，這類縮寫較為實用。例如 *en-US* 代表美式英文,而 *en-GB* 表示英式英文。

執行下列儲存格，以便在英式英文和義大利文間進行翻譯。

In [ ]:
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='it-IT', from_lang='en-GB')
print('{} -> {}'.format(text_to_translate,translation))

讓我們嘗試另一種翻譯 - 這次將美式英文翻譯為中文。

In [ ]:
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='zh-CN', from_lang='en-US')
print('{} -> {}'.format(text_to_translate,translation))

## 語音翻譯

您可以使用**語音**服務以翻譯語音語言。

現在您可以執行下列儲存格以建立和測試使用語音 SDK 翻譯聲音語音的函式。

In [ ]:
# 建立函數，以便將一種語言的音訊翻譯為另一種語言的文字
def translate_speech(cog_location, cog_key, audio_file=None, to_lang='fr-FR', from_lang='en-US'):
    from azure.cognitiveservices.speech import SpeechConfig, AudioConfig, ResultReason
    from azure.cognitiveservices.speech.translation import SpeechTranslationConfig, TranslationRecognizer

    # Configure the speech translation service
    translation_config = SpeechTranslationConfig(subscription=cog_key, region=cog_location)
    translation_config.speech_recognition_language = from_lang
    translation_config.add_target_language(to_lang)

    # Configure audio input
    if audio_file is None:
        audio_config = AudioConfig() # Use default input (microphone)
    else:
        audio_config = AudioConfig(filename=audio_file) # Use file input

    # Create a translation recognizer and use it to translate speech input
    recognizer = TranslationRecognizer(translation_config, audio_config)
    result = recognizer.recognize_once()

    # Did we get it?
    translation = ''
    speech_text = ''
    if result.reason == ResultReason.TranslatedSpeech:
        speech_text = result.text
        translation =  result.translations[to_lang]
    elif result.reason == ResultReason.RecognizedSpeech:
        speech_text = result.text
        translation =  'Unable to translate speech'
    else:
        translation = 'Unknown'
        speech_text = 'Unknown'

    # rturn the translation
    return speech_text, translation
    

# Test the function
import os

file_name = 'english.wav'
file_path = os.path.join('data', 'translation', file_name)
speech, translated_speech = translate_speech(cog_location, cog_key, file_path, to_lang='es', from_lang='en-US')
result = '{} -> {}'.format(speech, translated_speech)

# Show translated text
print(result)

注意，必須使用 2 個字元的語言程式碼來識別目標語言 (例如 *en*)，而源語言必須包括文化指示 (例如 *en-US*)。

讓我們嘗試將法文翻譯為英文。

In [ ]:
import os

file_name = 'french.wav'
file_path = os.path.join('data', 'translation', file_name)
speech, translated_speech = translate_speech(cog_location, cog_key, file_path, to_lang='en', from_lang='fr-FR')
result = '{} -> {}'.format(speech, translated_speech)

# Show translated text
print(result)

## 了解更多資訊

您可以在服務文件中了解關於[翻譯工具文字](https://docs.microsoft.com/azure/cognitive-services/translator/)和[使用語音服務翻譯](https://docs.microsoft.com/azure/cognitive-services/speech-service/index-speech-translation)的更多資訊。